# Jersey City

In [36]:
import pandas as pd 
import numpy as np
from string import punctuation
import re
import sys
import os
import pickle

from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
geolocator = Nominatim(user_agent="myGeolocator", timeout=2)

# loading saved objects into dataframe
errors_df = pd.read_pickle('errors_df.pkl')
df = pd.read_pickle('jersey_city.pkl')

# dataframe for public housing:
# eventually, these dataframes will be joined in someway which preserves both property types
# until then I will be dealing with them separately
# public_housing_df = pd.read_csv('JerseyCity_PublicHousing.csv')
public_housing_df = pd.read_pickle('public_housing_df.pkl')


In [18]:
raw_df = pd.read_csv('/Users/kylereaves/Downloads/0906demo191023.csv').convert_dtypes(infer_objects=True)

/usr/local/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (20,22,23,25,42,43,47,87,89) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [63]:
raw_df

,Municipality,Block,Lot,Qual,Property Location,Property Class,Owner's Name,Owner's Mailing Address,City/State/Zip,Sq. Ft.,...,Assessed.3,Latitude,Longitude,Neigh,VCS,StyDesc,Style,Join,Unnamed: 91,owner_zip
0,906,101.0,1.01,HM,1075 SECAUCUS RD.,1,"20 AQUARIUM DR.,LLC%NAT'L RETAIL",611 ROUTE 46 WEST,"HASBROUCK HEIGHTS, N.J. 07604",0,...,20733400,0,0,<NA>,<NA>,<NA>,<NA>,00101 00001 01HM M,<NA>,07604
1,906,101.0,2.0,HM,1045 SECAUCUS RD.,4A,PUBLIC SERVICE ELECTRIC & GAS CO.,80 PARK PL.TAX DEPT.T-6B,"NEWARK, N.J. 07102",0,...,227600,0,0,<NA>,<NA>,<NA>,<NA>,00101 00002 HM M,<NA>,07102
2,906,101.0,3.0,HM,SECAUCUS ROAD,15C,UNITED STATES OF AMERICA,"475 L'ENFANT PLAZA, SW","WASHINGTON, D C 20260",0,...,4182300,0,0,<NA>,<NA>,<NA>,<NA>,00101 00003 HM M,<NA>,20260
3,906,101.0,5.0,HM,INSIDE SECAUCUS RD.,4A,HOLLAND CO.C/O PUB.SER.GAS&ELEC.CO.,80 PARK PL.-TAX DEPT.T-6B,"NEWARK, N.J. 07102",0,...,682400,0,0,<NA>,<NA>,<NA>,<NA>,00101 00005 HM M,<NA>,07102
4,906,101.0,6.0,HM,SECAUCUS RD,15C,UNITED STATES OF AMERICA,"475 L'ENFANT PLAZA, SW","WASHINGTON, D C 20260",0,...,7122400,0,0,<NA>,<NA>,<NA>,<NA>,00101 00006 HM M,<NA>,20260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63948,906,99990.0,1.0,<NA>,VARIOUS,6A,VERIZON-NJ C/O DUFF & PHELPS,P.O. BOX 2749,"ADDISON, TX 75001",0,...,64373216,0,0,<NA>,<NA>,<NA>,<NA>,99990 00001 M,<NA>,75001
63949,906,99990.0,2.0,<NA>,VARIOUS,1,"TEXAS EASTERN TRANSMISSION,LP",P.O. BOX 2629,"ADDISON, TX 75001",0,...,22067400,0,0,<NA>,<NA>,<NA>,<NA>,99990 00002 M,<NA>,75001
63950,906,99990.0,3.0,<NA>,VARIOUS,15F,"AIRBNB, INC",888 BRANNAN ST.,"SAN FRANCISCO, CA 94103",0,...,0,0,0,<NA>,<NA>,<NA>,<NA>,99990 00003 M,<NA>,94103
63951,906,99990.0,4.0,<NA>,VARIOUS,4B,TRANSCONTINENTAL GAS PIPE LINE,"P.O. BOX 2400, MD 46-4","TULSA, OK 74102",0,...,2286000,0,0,<NA>,<NA>,<NA>,<NA>,99990 00004 M,<NA>,74102


In [37]:
raw_df['owner_zip'] = [elem[-1] for elem in raw_df['City/State/Zip'].str.split('  ')]

In [62]:
[elem[-1] for elem in raw_df['City/State/Zip'].str.split('  ')]

['07604',
 '07102',
 '20260',
 '07102',
 '20260',
 '07302',
 '20260',
 '20260',
 '20260',
 '07102',
 '07102',
 '07102',
 '20260',
 '07102',
 '07604',
 '60523',
 '07102',
 '07102',
 '07102',
 '07102',
 '60523',
 '07307',
 '07307',
 '07307',
 '07307',
 '07307',
 '07307',
 '07307',
 '07662',
 '07307',
 '08625',
 '07307',
 '08625',
 '07302',
 '07307',
 '07307',
 '07307',
 '07307',
 '07307',
 '07307',
 '07307',
 '07307',
 '07307',
 '07307',
 '07652',
 '07047',
 '07307',
 '07307',
 '07307',
 '07307',
 '07013',
 '07410',
 '07307',
 '07307',
 '07307',
 '08625',
 '07302',
 '07856',
 '07307',
 '07307',
 '07307',
 '07307',
 '07094',
 '07307',
 '08807',
 '07307',
 '07307',
 '07302',
 '07307',
 '07307',
 '07307',
 '07307',
 '07307',
 '07307',
 '07307',
 '07307',
 '07082',
 '07306',
 '07307',
 '07307',
 '07307',
 '07307',
 '07307',
 '07307',
 '07304',
 '07047',
 '07307',
 '07022',
 '07307',
 '07307',
 '07307',
 '07307',
 '07307',
 '07307',
 '07307',
 '07307',
 '07302',
 '07307',
 '07302',
 '07039',


In [4]:
# geo_zip_dict = {}
geo_zip_dict = pickle.load(open('geo_zip_dict.pickle', 'rb'))

In [10]:
geo_errors_dict = {}
for index, key in enumerate(geo_zip_dict.keys()):
    try:
        int(key)
        # replace key in the dictionary with its converted integer
    except ValueError:
        print(f"{key} at position {index} cannot be converted to an integer.")


NJ at position 9 cannot be converted to an integer.
 , NJ at position 32 cannot be converted to an integer.
NY at position 42 cannot be converted to an integer.
 J. at position 45 cannot be converted to an integer.
, FLORDA at position 62 cannot be converted to an integer.
 NJ at position 63 cannot be converted to an integer.
NEW YORK at position 144 cannot be converted to an integer.
N.Y. at position 158 cannot be converted to an integer.
CT at position 204 cannot be converted to an integer.
, N.J. at position 226 cannot be converted to an integer.
, NEW JERSEY at position 231 cannot be converted to an integer.
NC at position 307 cannot be converted to an integer.
 . at position 312 cannot be converted to an integer.
 , N.J. at position 323 cannot be converted to an integer.
, NJ at position 331 cannot be converted to an integer.
 , N.Y. at position 353 cannot be converted to an integer.
 CT. at position 362 cannot be converted to an integer.
CA at position 365 cannot be converted to 

In [146]:
for zipcode in raw_df.owner_zip:
    if zipcode not in geo_zip_dict.keys():
        # gelocate the zipcode
        geolocated_value = geolocator.geocode(zipcode)
        geo_zip_dict[zipcode] = geolocated_value
        print(f'{zipcode} added to geo_zip_dict')
    else:
        print(f'{zipcode} already in geo_zip_dict…skipping…')


07604 added to geo_zip_dict
07102 added to geo_zip_dict
20260 added to geo_zip_dict
07102 already in geo_zip_dict
20260 already in geo_zip_dict
07302 added to geo_zip_dict
20260 already in geo_zip_dict
20260 already in geo_zip_dict
20260 already in geo_zip_dict
07102 already in geo_zip_dict
07102 already in geo_zip_dict
07102 already in geo_zip_dict
20260 already in geo_zip_dict
07102 already in geo_zip_dict
07604 already in geo_zip_dict
60523 added to geo_zip_dict
07102 already in geo_zip_dict
07102 already in geo_zip_dict
07102 already in geo_zip_dict
07102 already in geo_zip_dict
60523 already in geo_zip_dict
07307 added to geo_zip_dict
07307 already in geo_zip_dict
07307 already in geo_zip_dict
07307 already in geo_zip_dict
07307 already in geo_zip_dict
07307 already in geo_zip_dict
07307 already in geo_zip_dict
07662 added to geo_zip_dict
07307 already in geo_zip_dict
08625 added to geo_zip_dict
07307 already in geo_zip_dict
08625 already in geo_zip_dict
07302 already in geo_zip_d

In [153]:
pickle.dump(geo_zip_dict, open("geo_zip_dict.pickle", "wb"))

In [139]:
len(pd.Series(raw_df.owner_zip).unique()), pd.Series(raw_df.owner_zip).unique()

(1730,
 array(['07604', '07102', '20260', ..., '60690', '10066', '74102'],
       dtype=object))

In [87]:
raw_df.columns

Index(['Municipality', 'Block', 'Lot', 'Qual', 'Property Location',
       'Property Class', 'Owner's Name', 'Owner's Mailing Address',
       'City/State/Zip', 'Sq. Ft.', 'Yr. Built', 'Building Class',
       'Prior Block', 'Prior Lot', 'Prior Qual', 'Updated', 'Zone', 'Account',
       'Mortgage Account', 'Bank Code', 'Sp Tax Cd', 'Sp Tax Cd.1',
       'Sp Tax Cd.2', 'Sp Tax Cd.3', 'Map Page', 'Additional Lots',
       'Land Desc', 'Building Desc', 'Class 4 Code', 'Acreage', 'EPL Own',
       'EPL Use', 'EPL Desc', 'EPL Statute', 'EPL Init', 'EPL Further',
       'EPL Facility Name', 'Taxes 1', 'Taxes 2', 'Taxes 3', 'Taxes 4',
       'Sale Date', 'Deed Book', 'Deed Page', 'Sale Price', 'NU Code', 'Ratio',
       'Type/Use', 'Year', 'Owner', 'Street', 'City/State/Zip.1',
       'Land Assmnt', 'Building Assmnt', 'Exempt', 'Total Assmnt', 'Assessed',
       'Year.1', 'Owner.1', 'Street.1', 'City/State/Zip.2', 'Land Assmnt.1',
       'Building Assmnt.1', 'Exempt.1', 'Total Assmnt.1', 'As

In [85]:
raw_df['Type/Use'].unique()

array([nan, 1.0, 42.0, 43.0, 10.0, '1', 'Q2', '42', '10'], dtype=object)

In [83]:
raw_df.Latitude.unique()

array([0])

In [84]:
raw_df['Building Class'].unique()

array([nan, 'C', '47', '2', 'G', '0', '44', '43', '81', '87', '55', '38',
       'C041', 'C044', 'C045', 'C047', 'C046', '16', 'C068', 'C073',
       'C061', 'C062', 'C071', 'C097', 'C066', 'C065', 'S', 'C058',
       'C056', 'C059', 'C074', 'C077', '1', '.', 'C079', 'C078', 'C112',
       'C111', 'C076', 'C158', 'C156', 'C094', 'C153', 'C177', 'C053',
       'C105', 'C015', 'C013', 'C060', 'C091', 'C072', 'C110', 'C092',
       'C082', 'C093', 'C085', 'C067', '39', 'C090', 'C102', 'C080',
       'C203', '23', 'C031', 'C051', 'C043', 'C517', 'R', 'C052', 'C055',
       'C050', 'C107', 'C083', 'C081', 'C057', 'C213', 'C063', 'C017',
       'C100', ')', 'C109', 'C000', 'C042', 'C048', 'C036', 'C037',
       'C123', 'C139', 'C149', 'C113', 'C132', 'C108', 'C089', 'C114',
       'C103', 'C106', 'C137', 'C133', 'C152', 'C096', 'C180', 'H', '17',
       '3', '18'], dtype=object)

In [82]:
raw_df.Style.unique()

array([nan, 'CL', 'S', 'C', 'NU', 'T', 'D', 'VT', '20'], dtype=object)

In [79]:
raw_df.Zone.unique()

array([nan, 'I-2', 'R-1', 'R-2', 'R2', 'R-4', 'C-2', 'I-1', 'R-3', 'C-1',
       '-2', 'I-3', 'P', '64', 'C-4', 'R-3A', 'C-3'], dtype=object)

In [74]:
raw_df['Property Class'].unique()

array(['1', '4A', '15C', '4B', '2', '15F', '4C', '15D', '5A', '5B', '15A',
       '15B', '15E', '6A'], dtype=object)

In [72]:
raw_df.columns[4]

'propertyLocation'

In [40]:
raw_df.columns[20], raw_df.columns[23], raw_df.columns[25], raw_df.columns[42], raw_df.columns[43], raw_df.columns[47], raw_df.columns[87], raw_df.columns[89]

('Sp Tax Cd',
 'Sp Tax Cd.3',
 'Additional Lots',
 'Deed Book',
 'Deed Page',
 'Type/Use',
 'VCS',
 'Style')

In [6]:
# appending the path to access my helper functions
sys.path.append('/Users/kylereaves/src/landlord_data/')

import helpers

In [4]:
list(df.columns)

['street_address',
 'owner_name',
 'owner_mailing_address',
 'city_state_zip',
 'owner_full_mailing_address',
 'property_full_address',
 'number_properties_owned',
 'units_at_property',
 'g_code',
 'latitude',
 'longitude',
 'list_properties_owned']

In [7]:
helpers.COLUMNS

['street_address',
 'owner_name',
 'owner_mailing_address',
 'city_state_zip',
 'owner_full_mailing_address',
 'property_full_address',
 'units_at_property',
 'g_code',
 'latitude',
 'longitude',
 'number_properties_owned',
 'list_properties_owned']

In [8]:
helpers.COLUMNS == list(df.columns)

False

In [3]:
def get_properties(owner):
    return df[df.owner_name == owner]['street_address'].tolist()

In [4]:
df['list_properties_owned'] = [get_properties(owner) for owner in df.owner_name]

In [15]:
df.city_state_zip.unique()

<StringArray>
[        'JERSEY CITY, N.J.  07307',           'JERSEY CITY, NJ  07307',
               'PARAMUS, NJ  07652',         'NORTH BERGEN,  NJ  07047',
               'CLIFTON, NJ  07013',             'FAIR LAWN, NJ  07410',
               'JERSEY CITY  07307',              'SECAUCUS, NJ  07094',
           'BRIDGEWATER, NJ  08807',           'JERSEY CITY, NJ  07302',
 ...
              'ORANGE, N.J.  07050',    'ROCKVILLE CENTRE, N.Y.  11570',
        'STATEN ISLAND,  NY  10309',          'STROUDSBURG, PA.  18360',
              'HORSHAM, PA.  19044',               'EDISION, NJ  08817',
          'DIAMOND BAR, CA.  91765', 'INDIRANAGAR BANGALORE IND  26102',
              'LOVELAND, OH  45140',          'NEW MILFORD , NJ  07606']
Length: 2892, dtype: string

In [8]:
# using convert_dtypes() to infer types from columns
df = df.convert_dtypes(infer_objects=True)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42030 entries, 0 to 42029
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   street_address              42030 non-null  string 
 1   owner_name                  42030 non-null  string 
 2   owner_mailing_address       42030 non-null  string 
 3   city_state_zip              42030 non-null  string 
 4   owner_full_mailing_address  42030 non-null  string 
 5   property_full_address       42030 non-null  string 
 6   number_properties_owned     42030 non-null  Int64  
 7   units                       42030 non-null  Int64  
 8   g_code                      42030 non-null  object 
 9   latitude                    42030 non-null  Float64
 10  longitude                   42030 non-null  Float64
 11  list_properties_owned       42030 non-null  object 
dtypes: Float64(2), Int64(2), object(2), string(6)
memory usage: 4.0+ MB


In [ ]:
# if the str doesn't contain certain substrings, 
# replace '&' with ','
# split at (',')
# if the list of strings is longer than 2, this implies there is more than one person listed on the deed, in which case
# a new list should be created, based on the previous sliced list
# the last name is the first str in the list; every following str should be joined in front of the first string; 
# that joined should then be appended to the list of title cased, cleaned names

In [56]:
df.owner_name.str.split()[0][1] + " " + df.owner_name.str.split()[0][0]

'PRADEEP PEDDI,'

In [69]:
df.owner_name[df.owner_name.str.contains("&")]

1               PAREJA, HENRY A. & MIRYAM C.
4                  MARTIN, BENIGNO & CARIDAD
5                     CHAWDA, KISHOR & BHANU
7                  RENVILL, ANGEL & GRISERDA
8                SIERRA, LUIS A. & CARMEN I.
                        ...                 
42016    NARASIMHADEVARA, S.S. &CHIVUKULA, D
42021      CALLONI, GIUSPPE & VITOLO, NICOLE
42022          TODDYWALLA, PHIROZE & SHEHNAZ
42025              BALANLAYOS, JESUS & MYRNA
42026     DANGCIL, PROCERFINA & WILLIAM A JR
Name: owner_name, Length: 17191, dtype: string

In [4]:
df.rename(columns={'propertyLocation': 'street_address',
                   'ownersName': 'owner_name',
                   'ownersMailingAddress': 'owner_mailing_address',
                   'cityStateZip': 'city_state_zip',
                   'ownersFullMailingAddress': 'owner_full_mailing_address',
                   'propertyFullAddress': 'property_full_address',
                   'propertiesOwned': 'number_properties_owned',
                   'gCode': 'g_code'
                   }, inplace=True)


In [5]:
df.rename(columns={'units': 'units_at_property'}, inplace=True)


In [5]:
df.reset_index(inplace=True, drop=True)

In [ ]:
df['number_properties_owned'] = [len(df.list_properties_owned.iloc[i]) for i in range(0, df.index[-1]+1)]

In [ ]:
len(df.list_properties_owned.iloc[-1])

In [ ]:
public_housing_df

In [ ]:
# public_housing_df['gCode'] = public_housing_df.propertyLocation.apply(geolocator.geocode)

# def getAssociatedProperties(manager_name=str):
#    return public_housing_df[public_housing_df.managerName == manager_name]['propertyLocation'].tolist()

# public_housing_df['associatedProperties'] = [getAssociatedProperties(manager) for manager in public_housing_df.managerName]
# public_housing_df['lat'] = [g.latitude for g in public_housing_df.gCode]
# public_housing_df['long'] = [g.longitude for g in public_housing_df.gCode]
# public_housing_df.to_pickle('public_housing_df.pkl')

### check to see if there are any addresses beyond 'Jersey City'
### if so, append them to errors_df

In [ ]:
df[df.g_code.str.contains('Jersey City') == False]

In [ ]:
errors_df

## helper functions

In [ ]:
# # number of times property appears in first column
# def propertyAppearances(address=str): 
#     return len(df[df.propertyLocation == address])
# #
# def getPropertiesOwned(owner=str):
#     return df[df.ownersName == owner]['propertyLocation'].unique().tolist()
# df['ownersFullMailingAddress'] = df['ownersMailingAddress'] + ', ' + df.cityStateZip
# df['propertyFullAddress'] = df.propertyLocation + ', ' + 'Jersey City, NJ'


In [ ]:
geocode_errors = [i for i,e in enumerate(errors_df.gCode) if e == None]

In [ ]:
errors_df.iloc[geocode_errors]

In [ ]:
jersey_city_replacement = {
    'TONNELE': 'TONNELLE',
    'MC ADOO': 'MCADOO',
    'M.L. KING DRIVE': 'MARTIN LUTHER KING DRIVE',
    'COLUMBUS': 'CHRISTOPHER COLUMBUS',
    'FIRST': '1st',
    'SECOND': '2nd',
    'THIRD': '3rd',
    'FOURTH': '4th',
    'FIFTH': '5th',
    'SIXTH': '6th',
    'SEVENTH': '7th',
    'EIGHTH': '8th',
    'NINETH': '9th',
    'TENTH': '10th',
    'NINTH': '9th',
    'FIFTEENTH': '15th',
    'SIXTEENTH': '16th',
    'MC DOUGALL': 'MCDOUGALL',
    'CARPENTIER': 'CARPENTER',
    'FOX HOUND': 'FOXHOUND',
    'SIEDLER': 'SEIDLER',
    'LIENAU': 'LINEAU',
    'KENNEDY': 'JOHN F KENNEDY',
    'LIBERTY ST.': 'LIBERTY AVE.',
    '-REAR': '',
    '(REAR,': '',
    '-FRONT': '',
    'HAMPTON CT.': 'HAMPTON COURT',
    'VARCK': 'VARICK',
    'THO': '',
    'AVE AVE': 'AVE',
    'TER': 'TR',
    '3633A': '3633',
    '3144A': '3144',
    'SHEARWATR': 'SHEARWATER',
    'PALUSL': "PAUL'S",
    'COURT TR': "CT",
    'EASTVIEW': "E VIEW",
    'M.L. KING DR.': 'MARTIN LUTHER KING DR.',
    ' 1ST FL.': '',
    ' SO.': '',
    ' N.': '',
    'ODGEN': 'OGDEN',
    'PATRSON': 'PATERSON',
    ' 61 COLES': '',
    'VAN HOUTEN ST.': 'VAN HOUTEN AVE.',
    'MC PHERSON ST.': 'MCPHERSON PL.',
    'MC PHERSON': 'MCPHERSON',
    'FIR ROAD': 'FIR ST.',
    'UNION AVE.': 'UNION ST.',
    'OLD BERGEN': 'OLD BERGEN AVE.',
    'AVE. AVE.': 'AVE.',
    'JACKSON AVE.': 'JACKSON ST.',
    '162 OLD BERGEN AVE. AVE.': '162 OLD BERGEN AVE.',
    'GLEN': 'GLENN',
    'GLENNN': 'GLENN',
    'DR.VE': 'DRIVE',
    '1 GLENNN LANE': '1 GLENN LANE',
    r'\(\,': '',
    r'-\d{2:}(.?\d{1}(A,?,?A?\b': '',
    r'-\d{2:}': '',
    r'\s#\d': '',
    'BUTTRNUT': 'BUTTERNUT',
    'CARPENTR': 'CARPENTER',
    '100 SHEARWATER CT.': '100 EAST SHEARWATER CT.',
    '78A RNE ST.': '78 THORNE ST.',
}


In [ ]:
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('TONNELE', 'TONNELLE')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('MC ADOO', 'MCADOO')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('M.L. KING DRIVE', 'MARTIN LUTHER KING DRIVE')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('COLUMBUS', 'CHRISTOPHER COLUMBUS')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('FIRST', '1st')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('SECOND', '2nd')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('THIRD', '3rd')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('FOURTH', '4th')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('FIFTH', '5th')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('SIXTH', '6th')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('SEVENTH', '7th')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('EIGHTH', '8th')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('NINETH', '9th')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('TENTH', '10th')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('NINTH', '9th')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('FIFTEENTH', '15th')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('SIXTEENTH', '16th')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('MC DOUGALL', 'MCDOUGALL')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('CARPENTIER', 'CARPENTER')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('FOX HOUND', 'FOXHOUND')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('SIEDLER', 'SEIDLER')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('LIENAU', 'LINEAU')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('KENNEDY', 'JOHN F KENNEDY')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('LIBERTY ST.', 'LIBERTY AVE.')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('-REAR', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('(REAR)', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('-FRONT', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('HAMPTON CT.', 'HAMPTON COURT')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('VARCK', 'VARICK')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('THO', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('AVE AVE', 'AVE')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('TER', 'TR')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('3633A', '3633')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('3144A', '3144')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('SHEARWATR', 'SHEARWATER')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('PALUSL', "PAUL'S")
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('COURT TR', "CT")
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('EASTVIEW', "E VIEW")
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('M.L. KING DR.', 'MARTIN LUTHER KING DR.')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace(' 1ST FL.', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace(' SO.', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace(' N.', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('ODGEN', 'OGDEN')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('PATRSON', 'PATERSON')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace(' 61 COLES', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('VAN HOUTEN ST.', 'VAN HOUTEN AVE.')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('MC PHERSON ST.', 'MCPHERSON PL.')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('MC PHERSON', 'MCPHERSON')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('FIR ROAD', 'FIR ST.')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('UNION AVE.', 'UNION ST.')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('OLD BERGEN', 'OLD BERGEN AVE.')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('AVE. AVE.', 'AVE.')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('JACKSON AVE.', 'JACKSON ST.')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('162 OLD BERGEN AVE. AVE.', '162 OLD BERGEN AVE.')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('GLEN', 'GLENN')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('GLENNN', 'GLENN')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('DR.VE', 'DRIVE')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('1 GLENNN LANE', '1 GLENN LANE')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace(r'\(\)', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace(r'-\d{2,}(.?\d{1}(A)?)?A?\b', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace(r'-\d{2,}', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace(r'\s#\d', '')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('BUTTRNUT', 'BUTTERNUT')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('CARPENTR', 'CARPENTER')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('100 SHEARWATER CT.', '100 EAST SHEARWATER CT.')
# errors_df.propertyLocation = errors_df.propertyLocation.str.replace('78A RNE ST.', '78 THORNE ST.')

# overwriting past mistakes
errors_df.propertyFullAddress = errors_df.propertyLocation + ' Jersey City, NJ'

In [ ]:
%%time
errors_df['gCode'] = errors_df.propertyFullAddress.apply(geolocator.geocode)

In [ ]:
geocode_errors = [i for i,e in enumerate(errors_df.gCode) if e == None]

In [ ]:
errors_df.iloc[geocode_errors].propertyLocation.unique()

Checking to see how many unique properties remain in the errors dataframe:

In [ ]:
len(errors_df.iloc[geocode_errors].propertyLocation.unique())

In [ ]:
geocode_errors = [i for i,e in enumerate(errors_df.gCode) if e == None]

In [ ]:
errors_df.iloc[geocode_errors]

In [ ]:
# reassigning the errors_df to the remaining errors
errors_df = errors_df.iloc[geocode_errors].copy()

In [ ]:
errors_df

In [ ]:
errors_df.to_pickle('errors_df.pkl')

In [ ]:
df

In [ ]:
df['latitude'] = [g.latitude for g in df.gCode]
df['longitude'] = [g.longitude for g in df.gCode]

In [ ]:
errors_df.propertyLocation.unique()

In [ ]:
errors_df['propertyFullAddress'] = errors_df.propertyLocation + ', Jersey City, NJ'

In [ ]:
%%time
errors_df['gCode'] = errors_df.propertyFullAddress.apply(geolocator.geocode)

In [12]:
# pickling the dataframe
df.to_pickle('jersey_city.pkl')

In [8]:
# exporting dataframe to csv, but indicating that i'm only exporting private property
df.to_csv('jersey_city_private_property.csv', index_label='property_id', columns=helpers.COLUMNS)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42030 entries, 0 to 42029
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   street_address              42030 non-null  string 
 1   owner_name                  42030 non-null  string 
 2   owner_mailing_address       42030 non-null  string 
 3   city_state_zip              42030 non-null  string 
 4   owner_full_mailing_address  42030 non-null  string 
 5   property_full_address       42030 non-null  string 
 6   number_properties_owned     42030 non-null  Int64  
 7   units_at_property           42030 non-null  Int64  
 8   g_code                      42030 non-null  object 
 9   latitude                    42030 non-null  Float64
 10  longitude                   42030 non-null  Float64
 11  list_properties_owned       42030 non-null  object 
dtypes: Float64(2), Int64(2), object(2), string(6)
memory usage: 4.0+ MB


In [17]:
[i for i in df.columns]

['street_address',
 'owner_name',
 'owner_mailing_address',
 'city_state_zip',
 'owner_full_mailing_address',
 'property_full_address',
 'number_properties_owned',
 'units',
 'g_code',
 'latitude',
 'longitude',
 'list_properties_owned']